In [ ]:
import os
print(os.listdir("/content"))


['.config', 'dogs-vs-cats', 'cats_dogs_light.zip', 'dogs-vs-cats.zip.zip', 'sample_data']


In [ ]:
from google.colab import files
import zipfile
import os

# Upload dataset (pick the file when prompted)
uploaded = files.upload()

# Check what file got uploaded
print("Files in /content:", os.listdir("/content"))

# Use the actual filename here (replace if different)
zip_path = "dogs-vs-cats.zip.zip"   # <<-- update if needed
extract_dir = "/content/dogs-vs-cats"

# Extract
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Confirm extracted folders
print("Extracted files:", os.listdir(extract_dir))


Saving dogs-vs-cats.zip.zip to dogs-vs-cats.zip (1).zip
Files in /content: ['.config', 'dogs-vs-cats.zip (1).zip', 'dogs-vs-cats', 'cats_dogs_light.zip', 'dogs-vs-cats.zip.zip', 'sample_data']
Extracted files: ['cats_dogs_light']


In [ ]:
# ================================
# STEP 2: Import Libraries
# ================================
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import os


In [ ]:
# ================================
# STEP 3: Load and Preprocess Data
# ================================
# Path to training images (inside cats_dogs_light/train)
data_dir = os.path.join(extract_dir, "cats_dogs_light", "train")
print("Training folder contains:", len(os.listdir(data_dir)), "images")

img_size = 64   # Resize all images to 64x64
X, y = [], []

for img_file in tqdm(os.listdir(data_dir)):
    label = 0 if "cat" in img_file else 1   # 0 = cat, 1 = dog
    img_path = os.path.join(data_dir, img_file)
    try:
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (img_size, img_size))
        X.append(img.flatten())
        y.append(label)
    except:
        pass

X = np.array(X)
y = np.array(y)
print("Dataset shape:", X.shape, y.shape)


Training folder contains: 1000 images


100%|██████████| 1000/1000 [00:01<00:00, 858.55it/s]

Dataset shape: (1000, 12288) (1000,)


In [ ]:
# ================================
# STEP 4: Train-Test Split & Scaling
# ================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Train set:", X_train.shape, " Test set:", X_test.shape)


Train set: (800, 12288)  Test set: (200, 12288)


In [ ]:
# ================================
# STEP 5: Train SVM Classifier
# ================================
svm = SVC(kernel="linear", C=1)  # try kernel="rbf" too
svm.fit(X_train, y_train)

# Predictions
y_pred = svm.predict(X_test)

# Evaluation
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


✅ Accuracy: 0.54

Confusion Matrix:
 [[48 43]
 [49 60]]

Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.53      0.51        91
           1       0.58      0.55      0.57       109

    accuracy                           0.54       200
   macro avg       0.54      0.54      0.54       200
weighted avg       0.54      0.54      0.54       200



In [ ]:
# ================================
# STEP 6: Test with a Sample Image
# ================================
def test_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (img_size, img_size))
    flat = img.flatten().reshape(1, -1)
    flat = scaler.transform(flat)
    prediction = svm.predict(flat)[0]
    label = "Dog 🐶" if prediction == 1 else "Cat 🐱"
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(label)
    plt.axis("off")
    plt.show()
